<a href="https://colab.research.google.com/github/muoyo/chicago-ridesharing/blob/master/notebooks/rideshare_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
!pip install sodapy

In [0]:
import numpy as np
import pandas as pd
import seaborn as sns
from sodapy import Socrata
import statsmodels.api as sm
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_squared_log_error
from sklearn.linear_model import LinearRegression
%matplotlib inline


In [100]:
#!/usr/bin/env python

# make sure to install these packages before running:
# pip install pandas
# !pip install sodapy

import pandas as pd
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
# client = Socrata("data.cityofchicago.org", None)

# Example authenticated client (needed for non-public datasets):
client = Socrata('data.cityofchicago.org',
                 'Tk6RhuGAFvF9P4ehsysybj3IW',
                 username="mokome@gmail.com",
                 password="Ch1cago!!")

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
client.timeout = 10000
results = client.get("m6dm-c72p", limit=2000000, select='trip_id, trip_start_timestamp, trip_end_timestamp, trip_seconds, trip_miles, pickup_community_area, dropoff_community_area, fare, tip, additional_charges, trip_total' )

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)
results_df

,trip_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_community_area,dropoff_community_area,fare,tip,additional_charges,trip_total
0,dd0cc1463ef161e24b5cfca43d21373135e63867,2019-01-16T16:15:00.000,2019-01-16T16:30:00.000,1346,8.21715513808,7,77,15,0,2.55,17.55
1,dd0cc1b0d94700b898009174fa407127565d41ea,2018-11-01T06:00:00.000,2018-11-01T07:00:00.000,3113,20.4166496394989,45,23,17.5,0,2.5,20
2,dd0cc1d54c5ab4f104fc20ce02a8534b686e1abb,2019-01-24T19:45:00.000,2019-01-24T19:45:00.000,469,1.5670652572791,31,31,5,0,2.55,7.55
3,dd0cc3d1ed0d5a9b4ca1b29b2e50a92fe3366dc1,2018-12-21T18:15:00.000,2018-12-21T18:30:00.000,880,4.1697883216,69,39,10,0,0,10
4,dd0cc46f48999e973c01cd45468f265e844666c8,2019-03-05T14:45:00.000,2019-03-05T15:00:00.000,1506,7.32213988795855,28,29,10,0,2.55,12.55
...,...,...,...,...,...,...,...,...,...,...,...
1999995,1143f37b3053e2f9d4f37f613cdd5825c26dda78,2019-08-12T01:45:00.000,2019-08-12T02:00:00.000,1015,6.722369,5,8,12.5,3,2.55,18.05
1999996,1143f38297b979a3b88696b2244bd02341d2a500,2019-07-19T09:15:00.000,2019-07-19T09:30:00.000,1306,7.388153,32,6,15,5,2.55,22.55
1999997,1143f48e90eb89cccbff886fc4147e3c5a27fa35,2019-09-03T16:15:00.000,2019-09-03T17:00:00.000,2786,14.704924,77,76,27.5,0,7.55,35.05
1999998,1143f5069330b99ec01ae6e2da8e24836719f852,2019-09-29T18:15:00.000,2019-09-29T18:45:00.000,1753,23.44713608976,76,NaN,27.5,0,9.14,36.64


In [111]:
columns_to_use = ['trip_id', 'trip_start_timestamp', 'trip_end_timestamp', 'trip_seconds',
       'trip_miles', 'pickup_community_area', 'fare', 'tip',
       'additional_charges', 'trip_total' ]

columns_to_drop = [ col for col in results_df.columns if col not in columns_to_use ]
df = results_df.drop(columns=columns_to_drop)

df['trip_start_timestamp'] = pd.to_datetime(df['trip_start_timestamp'])
df['trip_end_timestamp'] = pd.to_datetime(df['trip_end_timestamp'])
df['trip_seconds'] = df['trip_seconds'].fillna('0')
df['trip_seconds'] = df['trip_seconds'].astype('int64')

for col in ['trip_miles', 'fare', 'tip', 'additional_charges', 'trip_total']:
    df[col] = df[col].astype(float) 

df['start_weekday'] = df['trip_start_timestamp'].apply(lambda d: d.weekday())
df['start_hour'] = df['trip_start_timestamp'].apply(lambda d: d.hour)
df['start_time_block'] = df['start_hour'] // 3

df['start_date_plus_hour'] = df['trip_start_timestamp'].apply(lambda d: datetime(d.year, d.month, d.day, d.hour))

weather_df = pd.read_csv('https://github.com/muoyo/chicago-ridesharing/raw/master/data/chicago_weather.csv')
weather_df['hour'] = weather_df['hour'].apply(lambda x: '{:02d}'.format(x))
weather_df['start_date_plus_hour'] = pd.to_datetime(weather_df['date'] + ' ' + weather_df['hour'] + ':00:00')
weather_df = weather_df.rename(columns={'icon': 'precip'})

# def set_precip(precip):
#     if precip not in ['rain', 'snow']: 
#         precip = 'clear'

#     return precip

# weather_df['precip'] = weather_df['precip'].apply(set_precip)
precip_df = weather_df[['start_date_plus_hour', 'precip', 'apparentTemperature']]

df = df.merge(precip_df, how='left', on='start_date_plus_hour')
df.head()

,trip_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_community_area,fare,tip,additional_charges,trip_total,start_weekday,start_hour,start_time_block,start_date_plus_hour,precip,apparentTemperature
0,dd0cc1463ef161e24b5cfca43d21373135e63867,2019-01-16 16:15:00,2019-01-16 16:30:00,1346,8.217155,7,15.0,0.0,2.55,17.55,2,16,5,2019-01-16 16:00:00,clear,21.94
1,dd0cc1b0d94700b898009174fa407127565d41ea,2018-11-01 06:00:00,2018-11-01 07:00:00,3113,20.416650,45,17.5,0.0,2.50,20.00,3,6,2,2018-11-01 06:00:00,clear,50.02
2,dd0cc1d54c5ab4f104fc20ce02a8534b686e1abb,2019-01-24 19:45:00,2019-01-24 19:45:00,469,1.567065,31,5.0,0.0,2.55,7.55,3,19,6,2019-01-24 19:00:00,clear,-1.47
3,dd0cc3d1ed0d5a9b4ca1b29b2e50a92fe3366dc1,2018-12-21 18:15:00,2018-12-21 18:30:00,880,4.169788,69,10.0,0.0,0.00,10.00,4,18,6,2018-12-21 18:00:00,clear,21.29
4,dd0cc46f48999e973c01cd45468f265e844666c8,2019-03-05 14:45:00,2019-03-05 15:00:00,1506,7.322140,28,10.0,0.0,2.55,12.55,1,14,4,2019-03-05 14:00:00,clear,6.71


In [112]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2000000 entries, 0 to 1999999
Data columns (total 16 columns):
trip_id                  object
trip_start_timestamp     datetime64[ns]
trip_end_timestamp       datetime64[ns]
trip_seconds             int64
trip_miles               float64
pickup_community_area    object
fare                     float64
tip                      float64
additional_charges       float64
trip_total               float64
start_weekday            int64
start_hour               int64
start_time_block         int64
start_date_plus_hour     datetime64[ns]
precip                   object
apparentTemperature      float64
dtypes: datetime64[ns](3), float64(6), int64(4), object(3)
memory usage: 259.4+ MB


In [186]:
columns_to_use = ['apparentTemperature', 'start_weekday', 'start_hour', 'pickup_community_area']
columns_to_drop = [ col for col in df.columns if col not in columns_to_use ]
X = df.drop(columns=columns_to_drop)
X['apparentTemperature']=X['apparentTemperature'].fillna(X['apparentTemperature'].median())
X['pickup_community_area']=X['pickup_community_area'].fillna('0')

cont_cols = ['apparentTemperature']
cat_cols = [ col for col in columns_to_use if col not in cont_cols ]

enc = OneHotEncoder()

y = df['trip_total']

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

X_train_cont = X_train[cont_cols]
X_train_cat = X_train[cat_cols]
X_test_cont = X_test[cont_cols]
X_test_cat = X_test[cat_cols]


# Transform training and test sets
X_train_enc = enc.fit_transform(X_train_cat, y_train)
X_test_enc = enc.transform(X_test_cat)

# Convert these columns into a DataFrame 
columns = enc.get_feature_names(input_features=X_train_cat.columns)
X_train_cat = pd.DataFrame(X_train_enc.todense(), columns=columns, index=X_train.index)
X_test_cat = pd.DataFrame(X_test_enc.todense(), columns=columns, index=X_test.index)

# Combine categorical and continuous features into the final dataframe
X_train = pd.concat([X_train_cont, X_train_cat], axis=1)
X_test = pd.concat([X_test_cont, X_test_cat], axis=1)

X_train = sm.add_constant(X_train)
X_test = sm.add_constant(X_test)
model = sm.OLS(y_train,X_train).fit()
model.summary()

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             trip_total   R-squared:                       0.280
Model:                            OLS   Adj. R-squared:                  0.280
Method:                 Least Squares   F-statistic:                     5442.
Date:                Wed, 22 Jan 2020   Prob (F-statistic):               0.00
Time:                        16:58:44   Log-Likelihood:            -5.5607e+06
No. Observations:             1500000   AIC:                         1.112e+07
Df Residuals:                 1499892   BIC:                         1.112e+07
Df Model:                         107                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const                       11.3813      0.035    329.016      0.000      11.314      11.449
apparentTemperature          0.0166      0.001     32.122      0.000       0.016       0.018
start_weekday_0              1.7482      0.022     80.484      0.000       1.706       1.791
start_weekday_1              1.2498      0.022     56.799      0.000       1.207       1.293
start_weekday_2              1.5293      0.022     71.056      0.000       1.487       1.571
start_weekday_3              1.8857      0.021     91.832      0.000       1.845       1.926
start_weekday_4              1.7420      0.019     90.204      0.000       1.704       1.780
start_weekday_5              1.4667      0.019     77.676      0.000       1.430       1.504
start_weekday_6              1.7597      0.020     87.052      0.000       1.720       1.799
start_hour_0                -0.4615      0.042    -10.862      0.000      -0.545      -0.378
start_hour_1                -1.2468      0.050    -24.981      0.000      -1.345      -1.149
start_hour_2                -0.8245      0.059    -13.951      0.000      -0.940      -0.709
start_hour_3                 0.6522      0.070      9.282      0.000       0.514       0.790
start_hour_4                 4.3017      0.071     60.667      0.000       4.163       4.441
start_hour_5                 5.2825      0.062     85.303      0.000       5.161       5.404
start_hour_6                 3.6284      0.051     71.684      0.000       3.529       3.728
start_hour_7                 1.2119      0.041     29.289      0.000       1.131       1.293
start_hour_8                 0.2960      0.037      8.042      0.000       0.224       0.368
start_hour_9                -0.0398      0.037     -1.068      0.285      -0.113       0.033
start_hour_10                0.4247      0.039     10.924      0.000       0.348       0.501
start_hour_11                0.5843      0.038     15.336      0.000       0.510       0.659
start_hour_12                0.5372      0.037     14.476      0.000       0.464       0.610
start_hour_13                0.4060      0.037     10.966      0.000       0.333       0.479
start_hour_14                0.5928      0.037     16.089      0.000       0.521       0.665
start_hour_15                1.0492      0.036     29.368      0.000       0.979       1.119
start_hour_16                0.6293      0.035     18.187      0.000       0.561       0.697
start_hour_17                0.2454      0.033      7.503      0.000       0.181       0.309
start_hour_18               -0.5697      0.032    -17.962      0.000      -0.632      -0.508
start_hour_19               -1.0744      0.032    -33.132      0.000      -1.138      -1.011
start_hour_20               -1.2686      0.034    -37.215      0.000      -1.335      -1.202
start_hour_21 

In [157]:
X.loc[X['pickup_community_area'].isna()]

,pickup_community_area,apparentTemperature


In [188]:
# Fit the model
linreg = LinearRegression()
linreg.fit(X_train, y_train)

# Print R2 and MSE for training and test sets
print('Training r^2:', linreg.score(X_train, y_train))
print('Test r^2:', linreg.score(X_test, y_test))
print('Training MSE:', mean_squared_error(y_train, linreg.predict(X_train)))
print('Test MSE:', mean_squared_error(y_test, linreg.predict(X_test)))


Training r^2: 0.2796456916407004
Test r^2: 0.2793111555896206
Training MSE: 97.16699845357059
Test MSE: 96.31459561349644


In [110]:
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10223 entries, 0 to 10222
Data columns (total 22 columns):
date                    10223 non-null object
hour                    10223 non-null object
time                    10223 non-null int64
summary                 10223 non-null object
precip                  10223 non-null object
precipIntensity         10223 non-null float64
precipProbability       10223 non-null float64
precipType              7033 non-null object
temperature             10223 non-null float64
apparentTemperature     10223 non-null float64
dewPoint                10223 non-null float64
humidity                10223 non-null float64
pressure                10223 non-null float64
windSpeed               10223 non-null float64
windGust                10223 non-null float64
windBearing             10223 non-null int64
cloudCover              10223 non-null float64
uvIndex                 10223 non-null int64
visibility              10223 non-null float64
ozone     